# Sistema de Avaliação de Redações usando LangGraph

## Visão Geral
Este notebook apresenta um sistema automatizado de avaliação de redações implementado usando LangGraph e um modelo LLM. O sistema avalia redações com base em quatro critérios principais: relevância, gramática, estrutura e profundidade de análise.

## Motivação
Sistemas automatizados de avaliação de redações podem agilizar significativamente o processo de avaliação em ambientes educacionais, fornecendo avaliações consistentes e objetivas. Esta implementação visa demonstrar como modelos de linguagem grandes e fluxos de trabalho baseados em grafos podem ser combinados para criar um sistema sofisticado de avaliação.

## Componentes Principais
1. Grafo de Estado: Define o fluxo de trabalho do processo de avaliação
2. Modelo LLM: Fornece a compreensão e análise de linguagem subjacente
3. Funções de Avaliação: Funções separadas para cada critério de avaliação
4. Lógica Condicional: Determina o fluxo do processo de avaliação com base em pontuações intermediárias

## Método
O sistema segue uma abordagem passo a passo para avaliar redações:

1. Relevância do Conteúdo: Avalia quão bem a redação aborda o tema proposto
2. Verificação Gramatical: Avalia o uso da linguagem e a correção gramatical da redação
3. Análise da Estrutura: Examina a organização e o fluxo de ideias na redação
4. Profundidade de Análise: Mede o nível de pensamento crítico e insight apresentado

Cada etapa é executada condicionalmente com base nas pontuações das etapas anteriores, permitindo o término antecipado de redações de baixa qualidade. A pontuação final é uma média ponderada de todas as pontuações dos componentes individuais.

## Conclusão
Este notebook demonstra uma abordagem flexível e extensível para a avaliação automatizada de redações. Ao aproveitar o poder dos grandes modelos de linguagem e um fluxo de trabalho baseado em grafos, oferece uma avaliação nuançada de redações que considera múltiplos aspectos da qualidade da escrita. Este sistema pode ser refinado e adaptado para vários contextos educacionais, potencialmente melhorando a eficiência e consistência das avaliações de redações.


## Instalações e configurações

## Essa célula importa as bibliotecas necessárias e configura a chave da API da OpenAI.

In [32]:
!pip install langgraph
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
import re


# Carrega as variáveis de ambiente e configura a chave da API da OpenAI
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

## Definição do estado

Esta célula define a classe State, que representa o estado do processo de avaliação.

In [34]:
class State(TypedDict):
    """ Representa o estado do processo de avaliação da redação """
    essay: str
    relevance_score: float
    grammar_score: float
    structure_score: float
    depth_score: float
    final_score: float

## Inicialização do modelo de linguagem

Essa célula inicializa o modelo ChatOpenAI.

In [35]:
# Inicializa o modelo ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

## Grading Functions

This cell defines the functions used in the grading process, including score extraction and individual grading components.

In [37]:
def extract_score(content: str) -> float:
    """Extrai a pontuação numérica da resposta do LLM."""
    match = re.search(r'Pontuação:\s*(\d+(\.\d+)?)', content)
    if match:
        return float(match.group(1))
    raise ValueError(f"Não foi possível extrair a pontuação de: {content}")

def check_relevance(state: State) -> State:
    """Verifica a relevância da redação."""
    prompt = ChatPromptTemplate.from_template(
        "Analise a relevância da seguinte redação em relação ao tema dado, prezando pela excelência da Língua Portuguesa. "
        "Forneça uma pontuação de relevância entre 0 e 1. "
        "Sua resposta deve começar com 'Pontuação: ' seguida da pontuação numérica, "
        "depois forneça sua explicação.\n\nRedação: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["relevance_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Erro em check_relevance: {e}")
        state["relevance_score"] = 0.0
    return state

def check_grammar(state: State) -> State:
    """Verifica a gramática da redação."""
    prompt = ChatPromptTemplate.from_template(
        "Analise a gramática da Língua Portuguesa na seguinte redação. "
        "Forneça uma pontuação de gramática entre 0 e 1. "
        "Sua resposta deve começar com 'Pontuação: ' seguida da pontuação numérica, "
        "depois forneça sua explicação.\n\nRedação: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["grammar_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Erro em check_grammar: {e}")
        state["grammar_score"] = 0.0
    return state

def analyze_structure(state: State) -> State:
    """Analisa a estrutura da redação."""
    prompt = ChatPromptTemplate.from_template(
        "Analise a estrutura de acordo com a normal culta da Língua Portuguesa na seguinte redação. "
        "Forneça uma pontuação de estrutura entre 0 e 1. "
        "Sua resposta deve começar com 'Pontuação: ' seguida da pontuação numérica, "
        "depois forneça sua explicação.\n\nRedação: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["structure_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Erro em analyze_structure: {e}")
        state["structure_score"] = 0.0
    return state

def evaluate_depth(state: State) -> State:
    """Avalia a profundidade de análise na redação."""
    prompt = ChatPromptTemplate.from_template(
        "Avalie a profundidade de análise na seguinte redação. "
        "Forneça uma pontuação de profundidade entre 0 e 1. "
        "Sua resposta deve começar com 'Pontuação: ' seguida da pontuação numérica, "
        "depois forneça sua explicação.\n\nRedação: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["depth_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Erro em evaluate_depth: {e}")
        state["depth_score"] = 0.0
    return state

def calculate_final_score(state: State) -> State:
    """Calcula a pontuação final com base nas pontuações dos componentes individuais."""
    state["final_score"] = (
        state["relevance_score"] * 0.3 +
        state["grammar_score"] * 0.2 +
        state["structure_score"] * 0.2 +
        state["depth_score"] * 0.3
    )
    return state

## Definição do fluxo de trabalho

Essa célula define o fluxo de trabalho de avaliação usando StateGraph.

In [38]:
# Inicializa o StateGraph
workflow = StateGraph(State)

# Adiciona nós ao grafo
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("check_grammar", check_grammar)
workflow.add_node("analyze_structure", analyze_structure)
workflow.add_node("evaluate_depth", evaluate_depth)
workflow.add_node("calculate_final_score", calculate_final_score)

# Define e adiciona arestas condicionais
workflow.add_conditional_edges(
    "check_relevance",
    lambda x: "check_grammar" if x["relevance_score"] > 0.5 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "check_grammar",
    lambda x: "analyze_structure" if x["grammar_score"] > 0.6 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "analyze_structure",
    lambda x: "evaluate_depth" if x["structure_score"] > 0.7 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "evaluate_depth",
    lambda x: "calculate_final_score"
)

# Define o ponto de entrada
workflow.set_entry_point("check_relevance")

# Define o ponto de saída
workflow.add_edge("calculate_final_score", END)


# Compila o grafo
app = workflow.compile()

## Função de avaliação de redação

Essa célula define a função principal para avaliar uma redação usando o fluxo de trabalho definido.

In [39]:
def grade_essay(essay: str) -> dict:
    """Avalia a redação fornecida usando o fluxo de trabalho definido."""
    initial_state = State(
        essay=essay,
        relevance_score=0.0,
        grammar_score=0.0,
        structure_score=0.0,
        depth_score=0.0,
        final_score=0.0
    )
    result = app.invoke(initial_state)
    return result

## Redação de exemplo

Essa é uma redação de exemplo para testar o sistema de avaliação.

In [40]:
sample_essay = """
O Impacto dos Agentes de Inteligência Artificial e Fluxos de Trabalho Agentes com Multi-ferramentas na Sociedade Moderna


A Inteligência Artificial (IA) tem experimentado avanços significativos nas últimas décadas, e uma de suas manifestações mais revolucionárias é o desenvolvimento de agentes de IA e fluxos de trabalho agentes com multi-ferramentas. Estes agentes autônomos, capazes de aprender, adaptar-se e tomar decisões independentes, estão transformando a maneira como interagimos com a tecnologia e influenciando diversos setores da sociedade. Este ensaio explora os efeitos profundos desses agentes na sociedade moderna, discutindo seus benefícios, aplicações práticas e os desafios que apresentam.

No campo da saúde, os agentes de IA estão desempenhando um papel crucial na melhoria dos cuidados aos pacientes e na eficiência operacional das instituições médicas. Sistemas avançados como o DeepMind Health, do Google, utilizam agentes de IA para analisar imagens médicas, detectando sinais precoces de doenças como câncer e retinopatia diabética com precisão impressionante. Além disso, agentes com multi-ferramentas podem integrar dados de diferentes fontes, como registros eletrônicos de saúde, resultados de exames laboratoriais e informações genômicas, para oferecer diagnósticos mais precisos e planos de tratamento personalizados.

Durante a pandemia de COVID-19, agentes de IA foram empregados para modelar a propagação do vírus, prever surtos e otimizar a distribuição de recursos médicos. Esses agentes ajudaram governos e instituições de saúde a tomar decisões informadas sobre medidas de contenção e alocação de vacinas, demonstrando o potencial dos fluxos de trabalho agentes em situações de crise.

No setor financeiro, os agentes de IA estão redefinindo a análise de risco, detecção de fraude e gestão de investimentos. Robôs consultores (robo-advisors) utilizam algoritmos de IA para fornecer aconselhamento financeiro personalizado, considerando objetivos individuais, tolerância ao risco e condições de mercado. Empresas como a BlackRock e Vanguard incorporam agentes de IA em suas estratégias de investimento, permitindo decisões mais ágeis e baseadas em dados.

Além disso, agentes de IA com fluxos de trabalho agentes podem interagir com múltiplas ferramentas financeiras, monitorando transações em tempo real, analisando padrões de comportamento e identificando atividades suspeitas. Isso não apenas protege os consumidores, mas também fortalece a integridade do sistema financeiro global.

A indústria de transporte está passando por uma transformação significativa com a introdução de veículos autônomos e sistemas inteligentes de gerenciamento de tráfego. Agentes de IA são o cérebro por trás de carros autônomos, interpretando dados de sensores, câmeras e radares para tomar decisões de direção seguras. Empresas como Waymo, Uber e Tesla estão na vanguarda dessa tecnologia, prometendo reduzir acidentes causados por erro humano e melhorar a eficiência das viagens.

Em cidades inteligentes, agentes de IA gerenciam semáforos adaptativos, sistemas de transporte público e serviços de compartilhamento de veículos. Por exemplo, em Singapura, agentes inteligentes coordenam o fluxo de tráfego em tempo real, reduzindo congestionamentos e emissões de gases poluentes.

Na educação, agentes de IA estão promovendo uma revolução no ensino e aprendizado. Plataformas educacionais inteligentes utilizam agentes para adaptar currículos às necessidades individuais dos estudantes. Esses sistemas analisam o desempenho, identificam áreas de dificuldade e ajustam o conteúdo para otimizar o aprendizado.

Por exemplo, o sistema Knewton utiliza agentes de IA para fornecer recomendações personalizadas de estudo, enquanto o Squirrel AI, na China, oferece tutoria adaptativa para milhões de estudantes. Esses agentes permitem que os educadores atendam às necessidades específicas de cada aluno, promovendo um aprendizado mais eficaz e inclusivo.

Na indústria e manufatura, os agentes de IA estão otimizando processos de produção, manutenção preditiva e gerenciamento da cadeia de suprimentos. Robôs industriais equipados com agentes inteligentes podem adaptar-se a diferentes tarefas, aprender com erros e colaborar com trabalhadores humanos em ambientes de fábrica.

A Siemens, por exemplo, implementa agentes de IA em suas fábricas para monitorar o desempenho de máquinas, prever falhas e programar manutenções antes que ocorram interrupções. Isso resulta em maior eficiência, redução de custos e melhoria na qualidade dos produtos.

No setor de serviços, agentes de IA estão revolucionando o atendimento ao cliente. Chatbots e assistentes virtuais utilizam processamento de linguagem natural para interagir com clientes, resolver problemas comuns e encaminhar questões mais complexas a agentes humanos. Isso melhora a experiência do cliente e permite que as empresas atendam a um volume maior de consultas com eficiência.

Empresas como a Amazon e a Apple implementam agentes de IA em seus serviços de suporte, oferecendo assistência 24 horas por dia e personalizando as interações com base no histórico do cliente.

Apesar dos benefícios significativos, a adoção de agentes de IA e fluxos de trabalho agentes apresenta desafios complexos. A automação avançada ameaça substituir empregos em diversos setores, desde manufatura até serviços profissionais. Estudos do Fórum Econômico Mundial indicam que milhões de empregos podem ser afetados, exigindo políticas robustas de requalificação e educação para preparar a força de trabalho para novas oportunidades.

Questões éticas também são centrais neste debate. A tomada de decisões autônomas por agentes de IA levanta preocupações sobre responsabilidade legal em casos de falhas ou decisões prejudiciais. Além disso, algoritmos podem perpetuar vieses existentes se treinados em dados não representativos, levando a discriminação em áreas como contratação, crédito e justiça criminal.

A privacidade dos dados é outra área crítica. Agentes de IA dependem de grandes quantidades de dados pessoais para aprender e operar eficientemente. Garantir que esses dados sejam coletados e utilizados de maneira ética, respeitando regulamentos como a Lei Geral de Proteção de Dados (LGPD) no Brasil, é essencial para manter a confiança do público.

A crescente dependência de agentes de IA também levanta preocupações sobre segurança cibernética. Sistemas autônomos podem ser vulneráveis a ataques, manipulações e falhas sistêmicas. Incidentes envolvendo veículos autônomos ou sistemas financeiros automatizados podem ter consequências significativas, destacando a necessidade de protocolos de segurança robustos.

Além disso, há o risco de dependência excessiva da tecnologia, onde habilidades humanas podem ser negligenciadas ou perdidas. É importante equilibrar a adoção de agentes de IA com o desenvolvimento contínuo das capacidades humanas e o pensamento crítico.

O futuro dos agentes de IA é promissor, com avanços contínuos em áreas como processamento de linguagem natural, aprendizado por reforço e redes neurais profundas. A integração de agentes de IA com tecnologias emergentes como computação quântica e Internet das Coisas (IoT) ampliará ainda mais suas capacidades.

Iniciativas como o AutoGPT e o BabyAGI representam passos em direção a agentes de IA capazes de aprender e se adaptar de forma autônoma, executando tarefas complexas sem intervenção humana constante. Esses sistemas têm o potencial de revolucionar setores inteiros, mas também exigem uma consideração cuidadosa dos impactos sociais e éticos.

Em conclusão, os agentes de IA e fluxos de trabalho agentes com multi-ferramentas estão remodelando a sociedade moderna, oferecendo soluções inovadoras para desafios complexos e melhorando a eficiência em diversos setores. Seus benefícios são inegáveis, desde cuidados de saúde aprimorados até transportes mais seguros e educação personalizada.

No entanto, esses avanços vêm acompanhados de desafios significativos que exigem atenção cuidadosa. A sociedade deve abordar questões de emprego, ética, privacidade e segurança de dados para garantir que a integração de agentes de IA beneficie a todos de maneira equitativa.

À medida que avançamos para um futuro cada vez mais interconectado e orientado pela IA, é responsabilidade coletiva de governos, empresas, educadores e cidadãos trabalhar juntos. Devemos promover uma abordagem equilibrada que valorize a inovação tecnológica ao mesmo tempo em que protege os valores humanos fundamentais, assegurando que os agentes de IA sirvam como ferramentas para o bem-estar e progresso da humanidade.
    """

## Avaliação da redação de exemplo

Essa célula demonstra como usar o sistema de avaliação na redação de exemplo e exibir os resultados.

In [41]:
# Avalia a redação de exemplo
result = grade_essay(sample_essay)

# Converte as pontuações de 0-1 para 0-10
final_score = result['final_score'] * 10
relevance_score = result['relevance_score'] * 10
grammar_score = result['grammar_score'] * 10
structure_score = result['structure_score'] * 10
depth_score = result['depth_score'] * 10

# Exibe os resultados
print(f"Pontuação Final da Redação: {final_score:.2f}/10\n")
print(f"Pontuação de Relevância: {relevance_score:.2f}/10")
print(f"Pontuação de Gramática: {grammar_score:.2f}/10")
print(f"Pontuação de Estrutura: {structure_score:.2f}/10")
print(f"Pontuação de Profundidade: {depth_score:.2f}/10")

Pontuação Final da Redação: 8.65/10

Pontuação de Relevância: 9.50/10
Pontuação de Gramática: 8.50/10
Pontuação de Estrutura: 8.50/10
Pontuação de Profundidade: 8.00/10
